# Bulk processor and improved FlySight file and jump validation

Uses `./data` as a data lake.

In [38]:
%%capture
!x=$(pip list | awk '/haversine/') ; [[ -z "$x" ]] && make local

In [39]:
from pathlib import Path

import warnings

import bokeh.models as bm
import bokeh.plotting as bp
import ipywidgets as widgets
import pandas as pd

In [40]:
from ssscoring.calc import aggregateResults
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import getFlySightDataFromCSV
from ssscoring.calc import isValidMaximumAltitude
from ssscoring.calc import isValidMinimumAltitude
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import processJump
from ssscoring.calc import roundedAggregateResults
from ssscoring.calc import totalResultsFrom
from ssscoring.datatypes import JumpStatus
from ssscoring.constants import FT_IN_M
from ssscoring.flysight import getAllSpeedJumpFilesFrom
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

---
## Set DZ altitude MSL

Set the value in ft.

In [41]:

dropZones = {
    'Drop zone': [
        'Bay Area Skydiving',
        'Drop Zone Thailand',
        'Fehrbellin',
        'Mile High',
        'Neustadt-Glewe',
        'Paracaidismo Celaya',
        'Paraclete XP',
        'Saarlouis-Düren',
        'SkyDance SkyDiving',
        'Skydive Arizona',
        'Skydive Chicago',
        'Skydive Netheravon',
        'Skydive Puebla',
        'Skydive Saulgau',
        'Skydive Teuge',
        'Thai Sky Adventures',
    ],
    'Alt (ft)': [
        23.0,
        15.0,
        138.0,
        5500.0,
        115.0,
        5734.0,
        304.0,
        1119.0,
        100.0,
        1509.0,
        616.0,
        454.0,
        5744.0,
        1903.0,
        15.0,
        21.0,
    ],
}

pd.DataFrame(dropZones, columns=[ 'Drop zone', 'Alt (ft)', ])

,Drop zone,Alt (ft)
0,Bay Area Skydiving,23.0
1,Drop Zone Thailand,15.0
2,Fehrbellin,138.0
3,Mile High,5500.0
4,Neustadt-Glewe,115.0
5,Paracaidismo Celaya,5734.0
6,Paraclete XP,304.0
7,Saarlouis-Düren,1119.0
8,SkyDance SkyDiving,100.0
9,Skydive Arizona,1509.0


In [42]:
dropZoneAltMSL = 1509.0
ignoreBaseline = True

In [43]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">1509.00 ft</span> ( 459.95 m)<h1>')

In [44]:
jumpFiles = getAllSpeedJumpFilesFrom(Path('./data'))

---
## Process and detect non-jump file

The first file in the list for this test is a warm up file.  Detect and ignore it.

In [45]:
filePath = list(jumpFiles.keys())[0]
rawData, tag = getFlySightDataFromCSV(filePath)
data = convertFlySight2SSScoring(rawData, altitudeDZMeters=dropZoneAltMSLMeters)
jumpResults = processJump(data)
if jumpResults.status == JumpStatus.WARM_UP_FILE:
    display(widgets.HTML('<h3>%s - warm up file' % filePath.as_posix()))

HTML(value='<h3>data/09-11-00.CSV - warm up file')

---
## Process jump file


In [46]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpFilesList = list(jumpFiles.keys())
if (len(jumpFilesList) > 1):
    filePath = jumpFilesList[1]
    rawData, tag = getFlySightDataFromCSV(filePath)
    data = convertFlySight2SSScoring(rawData, altitudeDZMeters=dropZoneAltMSLMeters)
    jumpResult = processJump(data)
    if jumpResult.status == JumpStatus.OK:
        display(jumpResult.table)
        display(jumpResult.window)

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
10249,5.0,173.160,142.639183,50.52,231.14,12056.602757,224.344204
10274,10.0,316.908,116.481587,69.82,413.24,10913.188183,337.636847
10299,15.0,414.252,91.204833,77.58,551.40,9223.212014,424.173362
10324,20.0,452.988,54.542946,83.13,641.72,7221.628742,456.259861
10343,23.8,470.772,35.741311,85.66,671.23,5618.528158,472.126802


PerformanceWindow(start=np.float64(3806.884207022677), end=1707.0, validationStart=2713.0)

---
## Results

In [47]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpResults = processAllJumpFiles(jumpFiles=jumpFiles, altitudeDZMeters=dropZoneAltMSLMeters)
aggregate = aggregateResults(jumpResults)
aggregate

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 09-37-27:v1,464.85,173.160,316.908,414.252,452.988,470.772,23.8,470.772
data 10-56-45:v1,455.84,164.484,304.956,402.804,448.236,453.924,23.6,456.264
data 12-14-48:v1,454.19,175.644,312.732,407.484,444.276,460.188,22.8,460.188
data Ciurana-jump-02-container-09-56-08:v1,495.33,182.700,326.196,429.624,478.368,497.916,24.2,497.916


### Rounded results for training log

In [48]:
roundedResults = roundedAggregateResults(aggregate)
roundedResults

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 09-37-27:v1,465,173,317,414,453,471,23.8,471
data 10-56-45:v1,456,164,305,403,448,454,23.6,456
data 12-14-48:v1,454,176,313,407,444,460,22.8,460
data Ciurana-jump-02-container-09-56-08:v1,495,183,326,430,478,498,24.2,498


## All jumps

In [49]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)

In [50]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in jumpResults.keys():
    if ignoreBaseline and 'baseline' in resultRef:
        continue
    jumpResult = jumpResults[resultRef]
    if jumpResult.status == JumpStatus.OK:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.01f km/h</span></h1>' % ('green', resultRef, 'VALID', jumpResult.score)
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - %s</span></h1>' % ('red', resultRef, 'INVALID', jumpResult.status)

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.status == JumpStatus.OK:
        belowMaxAltitude = isValidMaximumAltitude(jumpResult.data.altitudeAGL.max())
        badJumpLegend = None
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            badJumpLegend = '<h3><span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude AGL was lower than the minimum scoring altitude according to IPC and USPA.</h3>'
        if not belowMaxAltitude:
            badJumpLegend = '<h3><span style="color: red"><span style="font-weight: bold">RE-JUMP:</span> exit altitude AGL exceeds the maximum altitude according to IPC and USPA.</h3>'
            validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - %s</span></h1>' % ('red', resultRef, 'INVALID', JumpStatus.ALTITUDE_EXCEEDS_MAXIMUM)
        display(widgets.HTML(validJumpStatus))            
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3?'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if badJumpLegend:
            display(widgets.HTML(badJumpLegend))
            if not belowMaxAltitude:
                continue
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips=[('Y-val', '@y{0.00}',),])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(
            individualPlot,
            jumpResult,
            lineColor=SPEED_COLORS[0])
        graphJumpResult(
            allJumpsPlot,
            jumpResult,
            lineColor=SPEED_COLORS[mixColor],
            legend='%s - %.2f' % (resultRef, jumpResult.score),
            showIt=False)

HTML(value='<hr><h1><span style="color: green">data 09-37-27:v1 jump - VALID - score = 464.9 km/h</span></h1>'…

HTML(value='<h3>Max speed = 471; exit at 3806 m (12489 ft), end scoring window at 1707 m (5600 ft)</h3?')

HTML(value='<h3><span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude AGL …

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,173,143,50.52,231.14,12057,224
,10.0,317,116,69.82,413.24,10913,338
,15.0,414,91,77.58,551.40,9223,424
,20.0,453,55,83.13,641.72,7222,456
,25.0,471,36,85.66,671.23,5619,472


HTML(value='<hr><h1><span style="color: green">data 10-56-45:v1 jump - VALID - score = 455.8 km/h</span></h1>'…

...

HTML(value='<h3><span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude AGL …

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,176,157,48.20,239.05,11551,236
,10.0,313,113,70.12,428.35,10410,333
,15.0,407,79,79.09,549.38,8742,415
,20.0,444,48,83.90,644.43,6781,447
,25.0,460,32,86.07,664.07,5620,461


HTML(value='<hr><h1><span style="color: red">data Ciurana-jump-02-container-09-56-08:v1 jump - INVALID - JumpS…

HTML(value='<h3>Max speed = 498; exit at 4308 m (14134 ft), end scoring window at 2052 m (6732 ft)</h3?')

HTML(value='<h3><span style="color: red"><span style="font-weight: bold">RE-JUMP:</span> exit altitude AGL exc…

---
## All skydives

In [51]:
sumResults = totalResultsFrom(aggregate)
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 09-37-27:v1,465,173,317,414,453,471,23.8,471
data 10-56-45:v1,456,164,305,403,448,454,23.6,456
data 12-14-48:v1,454,176,313,407,444,460,22.8,460
data Ciurana-jump-02-container-09-56-08:v1,495,183,326,430,478,498,24.2,498


,totalSpeed,meanSpeed,maxScore
totalSpeed,1870,467.5,495
